# Experiment C 

Ideas:

~~1) Estudar as diferentes distâncias para vetores 1-D em `scipy.spatial.distance`.~~

~~2) De 1, implementar e otimizar os que melhor performam comos portfólios.~~

3) Ver a possibilidade de recomendar com base em 2 ou mais empresas (`users`)

Eu não queria usar valores médios entre os vetores ou representantes de clusters,
pois creio que a distorção espacial ao calcular um vetor médio pode ser maior quanto
mais empresas eu tiver de entrada. Penso que uma abordagem interessante é que garante
pelo menos uma proximidade maior entre as empresas é por votação. Via clsuter e vetor
médio eu faço recomendações que não são as mais próximas de todos, mas nem tão distantes,
mas isso pode excluir os potenciais melhores recomendações. Com votação, eu aumento as 
chances de recomendar empresas que são muito próximas para umas, ainda que seja distante
de outras. 

4) Validar as recomendações com os portfólios.

Como utilizar os portfólios dados para validar o modelo? Não sabe-se a ordem em que os
portfólios foram crescendo. Então fazer um-a-um e N-a-N. Recomendações 1-para-1, 1-para-N, N-para-1 e N-para-N.

Estou pensando em uma métrica que trate de forma acumulada, p.e.:

- **A) 1-para-1:** Para cada uma empresa dentro de cada portfólio, eu vejo se a recomendação já está dentro do portfólio.

- **B) 1-para-N:** Para cada uma empresa dentro de cada portfólio, requisito N recomendações. Verifico quantas das N recomendações está no portfólio.

- **C) N-para-1:** Amostro N empresas e requisito 1 recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes.

- **D) N-para-N:** Amostro N empresas e requisito N recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes. 

4.1) E o desempate? Pensando...

Ideia: um misto entre o ranqueamento individual (1-para-N), mais votados e sorteio aleatório dos empatados por último.

Como abordar isso?

Posso fazer algo iterativo, tendo como objetivo obeter as $k$ recomendações solicitadas. Como o cálculo dos vizinhos é feito só uma vez, processamento não será um problema.

- **a)** Pego $L \times \text{div}(k,N_{in}) + \min(\text{mod}(k,N_{in}),1)$ recomendações por usuário de entrada.
- **b)** Ordeno por votação e desempato por ordem de proximidade individual.
- **c)** Se a lista ficar curta, faço $L \leftarrow L+1$ e volto para **a** até cehgar a $k$ recomendações.



*Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)*

In [1]:
%load_ext watermark

In [2]:
from typing import NewType, List
import functools
import operator

import numpy as np
import pandas as pd
from surprise import SVD, accuracy, Dataset, Reader
from scipy.spatial.distance import cosine

from loguru import logger
from tqdm import tqdm
from collections import Counter, defaultdict
from copy import deepcopy

In [3]:
# Run this cell before close.
%watermark
%watermark -p loguru
%watermark -p scipy
%watermark -p surprise
%watermark --iversion
%watermark -b -r -g

2020-06-23T09:15:36-03:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 9.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
loguru 0.5.1
scipy 1.5.0
surprise 0.1
numpy  1.19.0
pandas 1.0.5

Git hash: 1e8a51d89aec821c6a4118f019bfbff7e875509d
Git repo: https://github.com/ysraell/aceleradev_private.git
Git branch: master


In [5]:
path_data = '../data/'
top_cols = pd.read_csv('top_cols.csv')['cols'].to_list()
df_marked = pd.read_csv(path_data+'estaticos_market.csv', usecols=top_cols)
col_user = 'id'
top_cols.remove(col_user)

rest_cols = []
for col in top_cols:
    df_marked[col] = df_marked[col].fillna(0)*1

def normalize(x):
    return (x-np.min(x))/(np.max(x) - np.min(x)) if (np.max(x) - np.min(x)) > 0 else (x-np.min(x))

escala = 100
for col in top_cols:
    df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    
remove_cols = []
for col in top_cols:
    if df_marked[col].nunique() == 1:
        remove_cols.append(col)

df_marked = df_marked.drop(columns=remove_cols)
for col in remove_cols:
    top_cols.remove(col)

df_marked = pd.melt(df_marked, id_vars=["id"], var_name="itemID", value_name="rating").rename(columns={"id": "userID"})

reader = Reader(rating_scale=(0, escala))
#data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']].sample(frac=0.2), reader)
data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']], reader)
del df_marked

df_ep_list = [pd.read_csv(path_data+'estaticos_portfolio{}.csv'.format(i+1)) for i in range(3)]
tmp = []
for i in range(3):
    df_ep_list[i]['P'] = i+1 
    tmp.append(df_ep_list[i][['id','P']])
df_ep = pd.concat(tmp)
del df_ep_list
del tmp

In [6]:
Uid = NewType('uid', int)
Raw = NewType('raw', str)

def flat(a):
    return functools.reduce(operator.iconcat, a, [])

class ExSVD(SVD):
    """
        Classe extendida da surprise.SVD.
        
    
    """
    
    def __init__(self,**args):
        self.matrix_dict = {}
        super().__init__(**args)

    def fit(self,trainset: Dataset):
        """
            Reimplementei a SVD.fit para colocar um logger nível INFO.
        """
        logger.info("Treinando modelo SVD...")
        super().fit(trainset)
        logger.info("Pronto!")
    
    def _get_neighbors(self,uid: Uid, k: int = 1, black_list: List[Uid] = []):
        """
            Calcula todas as distâncias entre 'uid' de entrada e todos os outros 'uid'.
            A distância calciulada é armazenda e não calculada novamente. 
        """
        black_list.append(uid)
        k = k if k >= 0 else 0
        logger.info("Calculando todos os vizinhos...")
        for uid2 in tqdm(self.trainset.all_users()):
            ordered = tuple(sorted((uid,uid2)))
            if (uid2 not in black_list) and (ordered not in self.matrix_dict.keys()):
                self.matrix_dict[ordered] = cosine(self.pu[uid],self.pu[uid2])
        return [x[0] for x in sorted(
            [
                (uid2, self.matrix_dict[tuple(sorted((uid,uid2)))]) 
                for uid2 in self.trainset.all_users()
                if (uid2 not in black_list)
            ], key=lambda x: x[1])][:k-1]

    def _uid2raw(self, uid: Uid)-> str:
        '''
            uid -> raw.
            Valor interno para externo, o nome original do usuário.
        '''
        return self.trainset.to_raw_uid(uid)
    
    def _raw2uid(self, raw: Raw)-> int:
        '''
            raw -> uid.
            Valor externo para interno, o id interno do usuários..
        '''
        return self.trainset.to_inner_uid(raw)
    
    def recomender(self, in_list: List[Raw], k: int = 1, L: int = 3, Fk: int = 1, limit: int = 100)-> List[Raw]:
        '''
            Faz as recomendacoes.
            ##### Função incompleta #####
        '''
        # Pega quantas recomendações por usuário em `in_list`,
        # mas sem deixar faltar
        N_in = len(in_list)
        k = k if k > 0 else 1
        R_per_in = L*(k//N_in + min(k%N_in,1))

        # Pega os `uid`
        uid_in_list = []
        for raw in in_list:
            uid_in_list.append(self._raw2uid(raw))

        # Pega os vizinhos mais próximos de cada uid de entrada.
        done = False
        flag = True
        Rounds = 0
        while limit and (not done):
            Rounds += 1
            # Ele sempre pega todos novamente.
            recomendations_list = []
            for i,uid in enumerate(uid_in_list):
                logger.info("Calculando todos os vizinhos...{:,}/{:,} (Round: {:,}).".format(i+1,N_in,Rounds))
                recomendations_list.append(self._get_neighbors(uid,R_per_in,in_list))
            # Quando limit = 0, encerra.
            limit -= 1
            # Quando tem gente o suficiente, encerra.
            if len(set(flat(recomendations_list))) >= Fk*k:
                done = True
            # Depois do primeiro loop, pega um a mais.
            R_per_in += 1

        # Aqui gera um dicionário ordenando por votacao.
        count_rec = Counter(flat(recomendations_list)) # A votação!!
        count_rec = list(count_rec.items())
        ct_pos = defaultdict(list)
        #ct_pos_inv = defaultdict(list)
        while count_rec:
            tmp = count_rec.pop(0)
            ct_pos[tmp[1]].append(tmp[0])
            #ct_pos_inv[tmp[0]].append(tmp[1])

        # Aqui considera a posiçao de vizinhos mais proximos.
        #nn_pos = defaultdict(list)
        nn_pos_inv = defaultdict(list)
        tmp = deepcopy(recomendations_list)
        while tmp:
            tmp2 = tmp.pop(0)
            n = 0
            while tmp2:
                n += 1
                tmp3 = tmp2.pop(0)
                #nn_pos[n].append(tmp3)
                nn_pos_inv[tmp3].append(n)

        # Vai separando por votação e ordem de proximidade como desempate.      
        votos_list = list(ct_pos.keys())
        out_uid = []
        while votos_list and k:
            votos = max(votos_list)
            votos_list.remove(votos)
            tmp = sorted([(tmp, min(nn_pos_inv[tmp])) for tmp in ct_pos[votos]], key=lambda x: x[1])
            while tmp and k:
                out_uid.append(tmp.pop(0)[0])
                k -= 1

        # converte para Raw e "joga fora".
        return [self._uid2raw(uid) for uid in out_uid]



In [7]:
ex_algo = ExSVD(n_factors=5, verbose=True)
ex_algo.fit(data.build_full_trainset())

2020-06-23 09:16:13.475 | INFO     | __main__:fit:22 - Treinando modelo SVD...


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


2020-06-23 09:17:55.142 | INFO     | __main__:fit:24 - Pronto!


In [8]:
ex_algo.recomender(df_ep['id'].head(10).tolist(),k=100)

2020-06-23 09:19:33.766 | INFO     | __main__:recomender:84 - Calculando todos os vizinhos...1/10 (Round: 1).
2020-06-23 09:19:33.767 | INFO     | __main__:_get_neighbors:33 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:18<00:00, 24649.33it/s]
2020-06-23 09:19:53.363 | INFO     | __main__:recomender:84 - Calculando todos os vizinhos...2/10 (Round: 1).
2020-06-23 09:19:53.365 | INFO     | __main__:_get_neighbors:33 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:19<00:00, 23875.61it/s]
2020-06-23 09:20:13.700 | INFO     | __main__:recomender:84 - Calculando todos os vizinhos...3/10 (Round: 1).
2020-06-23 09:20:13.701 | INFO     | __main__:_get_neighbors:33 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:18<00:00, 25075.05it/s]
2020-06-23 09:20:33.198 | INFO     | __main__:recomender:84 - Calculando todos os vizinhos...4/10 (Round: 1).
2020-06-23 09:20:33.200 | INFO     | __main__:_get_neighbors:33 - Calculando todos os viz

['d37003c6e5d299c3fcd45f36ec6c26d39770f1d52edc415583f283c05bdd3f26',
 'a7c7ac14e19dc139657bed330ad55a63abacd98ab5fe1162532d0c178da38655',
 '88c111a406fb237c11ac8c1a1540d1f93febc8ba1844b3719be50a177b11e7fd',
 'caf5521892571b4a79b9d605787902883833e96453c611716d0e0a6739f15acf',
 '2211fdbbff8b2b4460993eeeb2ebb9e3470dd0934342c47ea5ea1c783b44311d',
 '3581b11d0ae2a734f38818f6a6c2db964efc36fe4ee5e17f77004d821f0336c2',
 '00b3f85b1e27cd448b149b59cde485f5cef8386750f867c0e6f7859846e23d12',
 'c9cc5cb720616b3f47277dc86cd2ce6288d7f10b6766489a1f92a9cfa758bbbc',
 'b31c6c06ca9a3565a2117dbea699d66e091780d69451df318f118bb4fff248da',
 '71f6c1cefa219525a00e9ad96268b7f102739f2b89bf190ededed9f31b73d125',
 'f650e8a882be5e77823bbdf5b143830aad0df27a288ec0fd41e7ebc6b90b5279',
 '4f6055255fe37ebc18b25986319d305e4956b43ab261f9d262f8da0523521389',
 '071ad3bb37776e4c6f999a88c758dae2845d18fd6bebd5b59ff8a353888ddf59',
 'd77af6e18315e16d26cafc9f9a83570f7cd90c8a5c3d55e77605d1f2c0cec1b4',
 'bd526a431957fdfe0d263b752aef7f56

**Item 4 pronto!***  *...assumo que não testei muito =D*

Adicionei mais um parâmetro, mas que não entra na hiper parametrização ($F_k$).